In [29]:
import os
import re

# 生成带有标签的医案
def tag_record_generate(origin_content, key, import_directory_name):
     # 带标签的医案
    tag_record_name = os.path.join(import_directory_name, author_name + '-' + key + '.txt')
    import_content = "#start\n"

    dia_idx = origin_content.find('辨证：')
    if dia_idx != -1:
        end_idx = origin_content.find('\n', dia_idx)
        dia_str = origin_content[dia_idx+3:end_idx]
        dia_tag = dia_str.replace('。','')
        import_content = import_content + 'dia-' + dia_tag + '\n'

    cure_idx = origin_content.find('治法：')
    if cure_idx != -1:
        end_idx = origin_content.find('\n', cure_idx)
        cure_str = origin_content[cure_idx+3:end_idx]
        cure_tag = cure_str.replace('。','')
        cure_tag_list = cure_tag.split('，')
        for tag in cure_tag_list:
            import_content = import_content + 'cure-' + tag + '\n'

    
    import_content = import_content + 'diease-' + key + '\n'
    import_content = import_content + '#end\n'
    import_content = import_content + origin_content
    fp = open(tag_record_name, mode='w', encoding='utf-8')
    fp.write(import_content)
    fp.close()


directory_name = 'Raw Medical Record'
import_directory_name = 'Import Medical Record'
file_name = 'record.txt'

fp = open(file_name, 'r', encoding='utf-8')
whole_record_start = False
whole_record = ''
whole_record_map = {}
whole_record_name = ''

for line in fp.readlines():
    # 判断是否以'案'结尾
    length = len(line)
    if length > 1 and line[length-2] == '案':

        if '◎' in line:
            continue
        if whole_record_start == False:
            whole_record_start = True
            whole_record_name = line.replace('\n', '')
        else:
            # 存储上一个whole_record
            whole_record_map[whole_record_name] = whole_record
            whole_record_name = line.replace('\n', '')
            whole_record = ''
            
    # 删除不必要信息
    if '一名真正的名中医' in line:
        continue
    elif '熊继柏临证医案实录' in line:
        continue
    else:
        whole_record += line

# 匹配'案一'、'案二'等信息
record_pattern = r'案[\u4e00\u4e8c\u4e09\u56db\u4e94\u516d\u4e03\u516b\u4e5d\u5341]+'
dia_pattern = r'^辨证：*$'
cure_pattern = r'^治法：*$'
author_name = '熊继柏'

record_cnt = 0
for key in whole_record_map.keys():
    files = re.split(record_pattern, whole_record_map[key])
    if len(files) == 1: # 一个案例
        record_name = os.path.join(directory_name, key+'.txt')
        fp = open(record_name, mode='w', encoding='utf-8')
        fp.write(whole_record_map[key])
        fp.close()

        tag_record_generate(whole_record_map[key], key, import_directory_name)
        record_cnt += 1

    elif len(files) > 1: # 多个案例
        # 创建文件夹
        dir_path = os.path.join(directory_name, key)
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
        for i in range(1, len(files)):
            record_name = os.path.join(dir_path, key + '-' + str(i) + '.txt')
            fp = open(record_name, mode='w', encoding='utf-8')
            fp.write(files[i])
            fp.close()

            tag_record_generate(files[i], key, import_directory_name)
        record_cnt += len(files)

print('生成医案文件成功,共生成{}份医案!'.format(record_cnt))


生成医案文件成功,共生成362份医案!
